In [1]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
# ============================================================================
# CELL 1: State Management & Utilities
# ============================================================================
# Frame-based visual input system for RetroArch bridge
# ============================================================================

from pathlib import Path
import json
import numpy as np
import time
from collections import deque

BASE_PATH = Path("/Users/achal/Downloads/PokeTest")
ACTION_FILE = BASE_PATH / "action.json"
STATE_FILE = BASE_PATH / "game_state.json"
CAPTURE_CONFIG_FILE = BASE_PATH / "capture_config.json"

EXPECTED_STATE_DIM = 6
FRAME_DIM = 7200  # 60 x 40 x 3 (RGB)
FRAME_SHAPE = (40, 60, 3)

# State vector from bridge: [x, y, map_id, in_battle, menu_flag, direction]

def normalize_game_state(raw_state):
    """Normalize context state for learning."""
    if len(raw_state) < 6:
        return raw_state
    
    normalized = raw_state.copy()
    normalized[0] = raw_state[0] / 255.0  # x
    normalized[1] = raw_state[1] / 255.0  # y
    normalized[2] = np.clip(raw_state[2], 0, 255)  # map_id
    normalized[3] = 1.0 if raw_state[3] > 0 else 0.0  # battle flag
    normalized[4] = 1.0 if raw_state[4] > 0 else 0.0  # menu flag
    normalized[5] = int(raw_state[5]) % 4  # direction
    
    return normalized


def compute_derived_features(current, prev):
    """Extract temporal features (8D)"""
    if prev is None:
        return np.zeros(8)
    
    vel_x = current[0] - prev[0]
    vel_y = current[1] - prev[1]
    map_changed = 1.0 if abs(current[2] - prev[2]) > 0.5 else 0.0
    battle_started = 1.0 if current[3] > prev[3] else 0.0
    battle_ended = 1.0 if current[3] < prev[3] else 0.0
    menu_opened = 1.0 if current[4] > prev[4] else 0.0
    menu_closed = 1.0 if current[4] < prev[4] else 0.0
    direction_changed = 1.0 if current[5] != prev[5] else 0.0
    
    return np.array([vel_x, vel_y, map_changed, battle_started, battle_ended,
                     menu_opened, menu_closed, direction_changed])


class FrameProcessor:
    """Handles frame differencing and temporal stacking."""
    
    def __init__(self, stack_size=3):
        self.stack_size = stack_size
        self.frame_stack = deque(maxlen=stack_size)
        self.prev_frame = None
        self.frame_diff_history = deque(maxlen=100)
        
        # ROI weights: edges more important than center
        self._roi_weights = None
    
    def get_roi_weights(self, shape):
        """Create ROI weight mask - edges weighted higher than center."""
        if self._roi_weights is not None and self._roi_weights.shape[0] == shape[0]:
            return self._roi_weights
        
        h, w = shape[0], shape[1]
        weights = np.ones((h, w), dtype=np.float32)
        
        # Center region (player) - lower weight
        center_y, center_x = h // 2, w // 2
        margin_y, margin_x = h // 4, w // 4
        weights[center_y - margin_y:center_y + margin_y, 
                center_x - margin_x:center_x + margin_x] = 0.5
        
        # Edges - higher weight
        edge_size = 3
        weights[:edge_size, :] = 1.5  # top
        weights[-edge_size:, :] = 1.5  # bottom
        weights[:, :edge_size] = 1.5  # left
        weights[:, -edge_size:] = 1.5  # right
        
        # Normalize
        weights = weights / weights.mean()
        
        # Expand to RGB
        self._roi_weights = np.stack([weights] * 3, axis=-1).flatten()
        return self._roi_weights
    
    def compute_frame_difference(self, frame):
        """Compute difference from previous frame. Returns 0-1 score."""
        if self.prev_frame is None:
            self.prev_frame = frame.copy()
            return 1.0  # First frame = maximum novelty
        
        # Compute absolute difference
        diff = np.abs(frame - self.prev_frame)
        
        # Apply ROI weights
        weights = self.get_roi_weights(FRAME_SHAPE)
        weighted_diff = diff * weights
        
        # Mean difference (0-1 range since frames are normalized)
        diff_score = np.mean(weighted_diff)
        
        self.prev_frame = frame.copy()
        self.frame_diff_history.append(diff_score)
        
        return diff_score
    
    def get_avg_frame_diff(self):
        """Get average frame difference over recent history."""
        if not self.frame_diff_history:
            return 0.5
        return np.mean(self.frame_diff_history)
    
    def add_frame(self, frame):
        """Add frame to temporal stack."""
        self.frame_stack.append(frame.copy())
    
    def get_stacked_frames(self):
        """Get temporally stacked frames as single vector."""
        if len(self.frame_stack) == 0:
            return np.zeros(FRAME_DIM * self.stack_size)
        
        # Pad with copies of first frame if not enough frames yet
        frames = list(self.frame_stack)
        while len(frames) < self.stack_size:
            frames.insert(0, frames[0])
        
        return np.concatenate(frames)
    
    def get_current_frame(self):
        """Get most recent frame."""
        if len(self.frame_stack) == 0:
            return np.zeros(FRAME_DIM)
        return self.frame_stack[-1]


# Global frame processor
frame_processor = FrameProcessor(stack_size=3)


def build_learning_state(derived, frame, in_battle, use_stacking=False):
    """
    Build learning state from derived features and frame data.
    
    Args:
        derived: 8D temporal features
        frame: 7200D frame data (flattened 60x40x3)
        in_battle: battle flag
        use_stacking: if True, use stacked frames (3x larger)
    """
    if use_stacking:
        frame_data = frame_processor.get_stacked_frames()
    else:
        frame_data = frame
    
    state = np.concatenate([derived, frame_data])
    
    # Small noise for numerical stability
    noise = np.random.randn(len(state)) * 0.0001
    return state + noise


def read_game_state(max_retries=3):
    """
    Read game state from JSON file.
    
    Returns:
        context_state: normalized [x, y, map_id, in_battle, menu_flag, direction]
        frame_state: 7200D frame data
        frame_diff: how much frame changed from previous
        dead: death flag
        raw_position: (raw_x, raw_y) for tile tracking
    """
    if not STATE_FILE.exists():
        return (np.zeros(EXPECTED_STATE_DIM), np.zeros(FRAME_DIM), 
                0.0, False, (0, 0))

    for attempt in range(max_retries):
        try:
            with open(STATE_FILE, "r") as f:
                data = json.loads(f.read())
            
            raw = data.get("state", [])
            frame_raw = data.get("frame", [])
            dead = bool(data.get("dead", False))
            
            # Raw position before normalization
            raw_x = int(raw[0]) if len(raw) > 0 else 0
            raw_y = int(raw[1]) if len(raw) > 1 else 0
            raw_position = (raw_x, raw_y)

            # Normalize context state
            context_state = normalize_game_state(np.array(raw, dtype=float))
            
            # Process frame
            frame_state = np.array(frame_raw, dtype=float) if frame_raw else np.zeros(FRAME_DIM)
            
            break

        except (json.JSONDecodeError, ValueError):
            if attempt < max_retries - 1:
                time.sleep(0.001)
                continue
            return (np.zeros(EXPECTED_STATE_DIM), np.zeros(FRAME_DIM),
                    0.0, False, (0, 0))
        except Exception:
            return (np.zeros(EXPECTED_STATE_DIM), np.zeros(FRAME_DIM),
                    0.0, False, (0, 0))

    # Ensure correct dimensions
    if context_state.shape[0] < EXPECTED_STATE_DIM:
        context_state = np.pad(context_state, (0, EXPECTED_STATE_DIM - context_state.shape[0]))
    elif context_state.shape[0] > EXPECTED_STATE_DIM:
        context_state = context_state[:EXPECTED_STATE_DIM]
    
    if frame_state.shape[0] < FRAME_DIM:
        frame_state = np.pad(frame_state, (0, FRAME_DIM - frame_state.shape[0]))
    elif frame_state.shape[0] > FRAME_DIM:
        frame_state = frame_state[:FRAME_DIM]

    # Compute frame difference and add to stack
    frame_diff = frame_processor.compute_frame_difference(frame_state)
    frame_processor.add_frame(frame_state)

    return context_state, frame_state, frame_diff, dead, raw_position


def write_action(action_name):
    """Write action for bridge to execute."""
    if action_name:
        action_name = action_name.upper()
    
    try:
        with open(ACTION_FILE, "w") as f:
            json.dump({"action": action_name}, f)
            f.flush()
    except Exception as e:
        print(f"[ERROR] Failed to write action: {e}")


def write_capture_config(capture_rate=1, reason="default"):
    """
    Write capture config for bridge to read.
    
    Args:
        capture_rate: 1 = every frame, 2 = every 2nd frame, etc.
        reason: why this rate was chosen (for logging)
    """
    try:
        with open(CAPTURE_CONFIG_FILE, "w") as f:
            json.dump({
                "capture_rate": capture_rate,
                "reason": reason,
                "timestamp": time.time()
            }, f)
    except Exception as e:
        print(f"[ERROR] Failed to write capture config: {e}")

In [3]:
# ============================================================================
# CELL 2: Perceptron Classes
# ============================================================================

class Perceptron:
    def __init__(self, kind, action=None, group=None, entity_type=None):
        self.kind = kind
        self.action = action
        self.group = group
        self.entity_type = entity_type
        
        self.utility = 1.0
        self.weights = None
        
        self.eligibility_fast = 0.0
        self.eligibility_slow = 0.0
        
        self.familiarity = 0.0
        self.activation_history = deque(maxlen=10)
        
        self.learning_rate = 0.01
        self.prediction_errors = deque(maxlen=50)

    def ensure_weights(self, dim):
        if self.weights is None:
            self.weights = np.random.randn(dim) * 0.001
        elif len(self.weights) != dim:
            # Resize weights if dimension changed
            old_weights = self.weights
            self.weights = np.random.randn(dim) * 0.001
            # Copy over what we can
            min_dim = min(len(old_weights), dim)
            self.weights[:min_dim] = old_weights[:min_dim]

    def predict(self, state):
        self.ensure_weights(len(state))
        raw_activation = np.dot(self.weights, state)
        
        if self.kind == "entity":
            novelty_factor = 1.0 / (1.0 + np.sqrt(self.familiarity * 0.5))
            decayed_activation = raw_activation * novelty_factor
            self.activation_history.append(abs(raw_activation))
            return decayed_activation
        else:
            return raw_activation

    def adapt_learning_rate(self):
        if len(self.prediction_errors) >= 50:
            avg_error = np.mean(self.prediction_errors)
            
            if avg_error < 0.1:
                self.learning_rate = max(0.001, self.learning_rate * 0.99)
            elif avg_error > 0.5:
                self.learning_rate = min(0.05, self.learning_rate * 1.01)

    def update(self, state, error, gamma_fast=0.5, gamma_slow=0.95, stagnation=0.0):
        self.ensure_weights(len(state))
        
        self.eligibility_fast = gamma_fast * self.eligibility_fast + 1.0
        self.eligibility_slow = gamma_slow * self.eligibility_slow + 1.0
        
        self.adapt_learning_rate()
        
        fast_update = 0.7 * self.learning_rate * error * state * self.eligibility_fast
        slow_update = 0.3 * self.learning_rate * error * state * self.eligibility_slow
        self.weights += fast_update + slow_update

        if self.kind == "action":
            if error > 0.01:
                if stagnation > 0.5:
                    self.utility *= 0.97
                elif error > 0.2:
                    self.utility = min(self.utility * 1.02, 2.0)
                else:
                    self.utility *= 0.995
            
            if self.group == "move":
                self.utility = np.clip(self.utility, 0.1, 2.0)
            else:
                self.utility = np.clip(self.utility, 0.01, 2.0)
        
        if self.kind == "entity" and len(self.activation_history) > 0:
            recent_avg = np.mean(self.activation_history)
            if recent_avg > 0.1:
                self.familiarity += 0.03
        
        if self.kind == "entity":
            prediction = self.predict(state)
            self.prediction_errors.append(abs(prediction - error))


class ControlSwapPerceptron(Perceptron):
    def __init__(self):
        super().__init__(kind="control_swap")
        self.swap_history = deque(maxlen=100)
        self.confidence = 0.0
        
    def should_swap(self, state, movement_stagnation):
        if self.weights is None:
            return False, 0.0
        
        self.ensure_weights(len(state))
        swap_score = np.dot(self.weights, state)
        stagnation_factor = np.tanh(movement_stagnation / 5.0)
        combined_score = swap_score * 0.7 + stagnation_factor * 0.3
        
        return combined_score > 0.5, abs(combined_score)
    
    def record_swap_outcome(self, state, swapped, novelty_gained):
        self.swap_history.append((swapped, novelty_gained))
        
        if len(self.swap_history) >= 20:
            recent = list(self.swap_history)[-20:]
            successful = sum(1 for swap, nov in recent if swap and nov > 0.2)
            self.confidence = successful / 20.0

In [4]:
# ============================================================================
# CELL 3: Brain Class
# ============================================================================

class Brain:
    def __init__(self):
        self.perceptrons = []
        
        self.prev_learning_states = deque(maxlen=50)
        self.prev_context_states = deque(maxlen=10)
        self.last_positions = deque(maxlen=30)
        self.action_history = deque(maxlen=100)
        
        self.control_mode = "move"
        self.timestep = 0
        self.last_action = None
        self.last_direction = 0
        
        self.MOVE_UTILITY_FLOOR = 0.05
        self.INTERACT_UTILITY_FLOOR = 0.15
        
        # === PERSISTENT EXPLORATION MEMORY ===
        self.EXPLORATION_MEMORY_FILE = BASE_PATH / "exploration_memory.json"
        self.exploration_memory = {}
        self.current_map_id = None
        self.SAVE_INTERVAL = 100
        
        # Direction mapping
        self.DIRECTION_NAMES = {0: "DOWN", 1: "UP", 2: "LEFT", 3: "RIGHT"}
        self.DIRECTION_TO_INT = {"DOWN": 0, "UP": 1, "LEFT": 2, "RIGHT": 3}
        self.INT_TO_ACTION = {0: "DOWN", 1: "UP", 2: "LEFT", 3: "RIGHT"}
        self.DIRECTION_DELTAS_INT = {0: (0, 1), 1: (0, -1), 2: (-1, 0), 3: (1, 0)}
        self.ACTION_DELTAS = {"UP": (0, -1), "DOWN": (0, 1), "LEFT": (-1, 0), "RIGHT": (1, 0)}
        
        self.load_exploration_memory()
        
        # === ADAPTIVE FRAME CAPTURE ===
        self.capture_rate = 1  # 1 = every frame, 2 = every 2nd, etc.
        self.MIN_CAPTURE_RATE = 1
        self.MAX_CAPTURE_RATE = 5
        self.frame_diff_threshold_high = 0.05  # Big change
        self.frame_diff_threshold_low = 0.01   # Almost no change
        self.recent_frame_diffs = deque(maxlen=30)
        
        # === MODE SWAP THRESHOLDS ===
        self.DEFAULT_MOVE_TO_INTERACT_THRESHOLD = 15
        self.DEFAULT_INTERACT_TO_MOVE_THRESHOLD = 25
        self.move_to_interact_threshold = self.DEFAULT_MOVE_TO_INTERACT_THRESHOLD
        self.interact_to_move_threshold = self.DEFAULT_INTERACT_TO_MOVE_THRESHOLD
        self.frames_in_current_mode = 0
        self.position_at_mode_swap = None
        self.last_map_id = None
        self.last_battle_state = None
        
        # === STATE STAGNATION ===
        self.STATE_STAGNATION_THRESHOLD = 20
        self.state_stagnation_count = 0
        self.last_context_state_hash = None
        
        # === NOVELTY WEIGHTS ===
        self.UNVISITED_TILE_BONUS = 1.5
        self.OBSTRUCTION_PENALTY = 0.25
        
        # Multi-scale memory
        self.visited_maps = {}
        self.map_novelty_debt = {}
        self.location_memory = {}
        self.location_novelty = {}
        
        self.swap_perceptron = ControlSwapPerceptron()
        self.error_history = deque(maxlen=1000)
        self.frame_diff_history = deque(maxlen=1000)
        self.innate_entities_spawned = False
        
        # === REPETITION TRACKING ===
        self.consecutive_action_count = 0
        self.current_repeated_action = None
        self.PENALTY_THRESHOLD = 12

    # =========================================================================
    # ADAPTIVE CAPTURE RATE
    # =========================================================================
    
    def update_capture_rate(self, frame_diff, stagnation_level):
        """Adjust frame capture rate based on performance."""
        self.recent_frame_diffs.append(frame_diff)
        avg_diff = np.mean(self.recent_frame_diffs) if self.recent_frame_diffs else 0.5
        
        old_rate = self.capture_rate
        
        # High stagnation = need more frames
        if stagnation_level > 0.7 or self.state_stagnation_count > 10:
            self.capture_rate = self.MIN_CAPTURE_RATE  # Every frame
        # Big visual changes = need more frames
        elif avg_diff > self.frame_diff_threshold_high:
            self.capture_rate = max(self.MIN_CAPTURE_RATE, self.capture_rate - 1)
        # Very little change = can skip frames
        elif avg_diff < self.frame_diff_threshold_low and stagnation_level < 0.3:
            self.capture_rate = min(self.MAX_CAPTURE_RATE, self.capture_rate + 1)
        
        # Write config if changed
        if self.capture_rate != old_rate:
            reason = f"stag={stagnation_level:.2f}, diff={avg_diff:.3f}"
            write_capture_config(self.capture_rate, reason)
        
        return self.capture_rate

    # =========================================================================
    # EXPLORATION MEMORY
    # =========================================================================
    
    def load_exploration_memory(self):
        try:
            if self.EXPLORATION_MEMORY_FILE.exists():
                with open(self.EXPLORATION_MEMORY_FILE, 'r') as f:
                    data = json.load(f)
                    self.exploration_memory = {}
                    for map_key, map_data in data.items():
                        map_id = int(map_key.replace('map_', ''))
                        self.exploration_memory[map_id] = self._deserialize_map_memory(map_data)
                print(f"  Loaded exploration memory: {len(self.exploration_memory)} maps")
            else:
                self.exploration_memory = {}
        except Exception as e:
            print(f"  Error loading exploration memory: {e}")
            self.exploration_memory = {}

    def _deserialize_map_memory(self, map_data):
        return {
            'visited_tiles': set(tuple(t) for t in map_data.get('visited_tiles', [])),
            'obstructions': set(tuple(t) for t in map_data.get('obstructions', [])),
            'last_visited_timestep': map_data.get('last_visited_timestep', 0),
        }

    def save_exploration_memory(self):
        try:
            data = {}
            for map_id, map_data in self.exploration_memory.items():
                data[f'map_{map_id}'] = {
                    'visited_tiles': list(map_data['visited_tiles']),
                    'obstructions': list(map_data['obstructions']),
                    'last_visited_timestep': map_data['last_visited_timestep'],
                }
            with open(self.EXPLORATION_MEMORY_FILE, 'w') as f:
                json.dump(data, f)
        except Exception as e:
            print(f"  Error saving exploration memory: {e}")

    def get_current_map_memory(self, map_id):
        if map_id not in self.exploration_memory:
            self.exploration_memory[map_id] = {
                'visited_tiles': set(),
                'obstructions': set(),
                'last_visited_timestep': self.timestep,
            }
        return self.exploration_memory[map_id]

    def record_visited_tile(self, x, y, map_id):
        memory = self.get_current_map_memory(map_id)
        memory['visited_tiles'].add((int(x), int(y)))
        memory['last_visited_timestep'] = self.timestep

    def record_obstruction(self, x, y, map_id, direction):
        dx, dy = self.DIRECTION_DELTAS_INT.get(direction, (0, 0))
        memory = self.get_current_map_memory(map_id)
        memory['obstructions'].add((int(x + dx), int(y + dy)))

    # =========================================================================
    # STAGNATION DETECTION
    # =========================================================================
    
    def get_context_state_hash(self, context_state):
        return (round(context_state[0], 2), round(context_state[1], 2), 
                int(context_state[2]), int(context_state[3]),
                round(context_state[4], 2), int(context_state[5]))

    def check_state_stagnation(self, context_state):
        current_hash = self.get_context_state_hash(context_state)
        if current_hash == self.last_context_state_hash:
            self.state_stagnation_count += 1
        else:
            self.state_stagnation_count = 0
        self.last_context_state_hash = current_hash
        return self.state_stagnation_count >= self.STATE_STAGNATION_THRESHOLD

    # =========================================================================
    # MODE CONTROL
    # =========================================================================
    
    def determine_control_mode(self, context_state, raw_position=None):
        if context_state[3] > 0.5:  # In battle
            return "battle"
        
        self.frames_in_current_mode += 1
        position_stagnation = self.get_position_stagnation()
        
        # Check for state stagnation
        if self.check_state_stagnation(context_state):
            new_mode = "interact" if self.control_mode == "move" else "move"
            self.control_mode = new_mode
            self.position_at_mode_swap = (context_state[0], context_state[1])
            self.frames_in_current_mode = 0
            self.state_stagnation_count = 0
            return self.control_mode
        
        # Normal mode switching
        if self.control_mode == "move" and position_stagnation >= self.move_to_interact_threshold:
            self.control_mode = "interact"
            self.position_at_mode_swap = (context_state[0], context_state[1])
            self.frames_in_current_mode = 0
        elif self.control_mode == "interact" and self.frames_in_current_mode >= self.interact_to_move_threshold:
            self.control_mode = "move"
            self.position_at_mode_swap = (context_state[0], context_state[1])
            self.frames_in_current_mode = 0
        
        return self.control_mode

    # =========================================================================
    # HELPER METHODS
    # =========================================================================
    
    def add(self, p):
        self.perceptrons.append(p)

    def actions(self):
        return [p for p in self.perceptrons if p.kind == "action"]

    def entities(self):
        return [p for p in self.perceptrons if p.kind == "entity"]

    def get_position_stagnation(self):
        if len(self.last_positions) < 2:
            return 0
        current_pos = self.last_positions[-1]
        return sum(1 for pos in reversed(list(self.last_positions)[:-1]) if pos == current_pos)

    def get_group_weight(self, group):
        return sum(a.utility for a in self.actions() if a.group == group)

    def update_position(self, x, y):
        self.last_positions.append((int(x), int(y)))

    def stagnation_level(self, window=10):
        if len(self.prev_learning_states) < window:
            return 0.0
        recent = list(self.prev_learning_states)[-window:]
        diffs = [np.linalg.norm(recent[i] - recent[i-1]) for i in range(1, len(recent))]
        return 1.0 - np.tanh(np.mean(diffs) * 2.0)

    def track_consecutive_action(self, action_name):
        if action_name == self.current_repeated_action:
            self.consecutive_action_count += 1
        else:
            self.current_repeated_action = action_name
            self.consecutive_action_count = 1

    def enforce_utility_floors(self):
        for a in self.actions():
            floor = self.MOVE_UTILITY_FLOOR if a.group == "move" else self.INTERACT_UTILITY_FLOOR
            a.utility = max(a.utility, floor)

    # =========================================================================
    # LEARNING
    # =========================================================================
    
    def spawn_innate_entities(self, learning_state):
        if self.innate_entities_spawned:
            return
        # Create basic sensory entities
        for etype in ["sense_visual", "sense_movement", "sense_change"]:
            entity = Perceptron("entity", entity_type=etype)
            entity.ensure_weights(len(learning_state))
            self.add(entity)
        self.innate_entities_spawned = True

    def compute_error(self, state, next_state):
        """Compute prediction error between states."""
        # Derived features (first 8 dims) weighted more
        derived_diff = np.linalg.norm(next_state[:8] - state[:8]) * 5.0
        # Frame difference
        frame_diff = np.linalg.norm(next_state[8:] - state[8:])
        return derived_diff + frame_diff

    def learn(self, learning_state, next_learning_state, context_state, 
              next_context_state, frame_diff, dead=False, raw_position=None):
        """Main learning update."""
        
        # Ensure same dimensions
        if learning_state.shape != next_learning_state.shape:
            max_dim = max(len(learning_state), len(next_learning_state))
            learning_state = np.pad(learning_state, (0, max(0, max_dim - len(learning_state))))
            next_learning_state = np.pad(next_learning_state, (0, max(0, max_dim - len(next_learning_state))))
        
        if not self.innate_entities_spawned:
            self.spawn_innate_entities(learning_state)
        
        # Compute error
        error = self.compute_error(learning_state, next_learning_state)
        self.error_history.append(error)
        self.frame_diff_history.append(frame_diff)
        
        # Update exploration tracking
        current_map = int(context_state[2])
        if raw_position:
            self.record_visited_tile(raw_position[0], raw_position[1], current_map)
        
        # Detect map change
        if self.current_map_id is not None and current_map != self.current_map_id:
            self.save_exploration_memory()
            self.control_mode = "move"
            self.frames_in_current_mode = 0
            print(f"  🗺️ MAP CHANGE → {current_map}")
        self.current_map_id = current_map
        
        # Update all perceptrons
        stagnation = self.stagnation_level()
        for p in self.perceptrons:
            p.update(learning_state, error, stagnation=stagnation)
        
        self.enforce_utility_floors()
        
        # Update adaptive capture rate
        self.update_capture_rate(frame_diff, stagnation)
        
        # Periodic save
        if self.timestep % self.SAVE_INTERVAL == 0:
            self.save_exploration_memory()
        
        # Log state
        self.action_history.append(self.last_action)

    def log_state(self, learning_state, context_state):
        self.prev_learning_states.append(learning_state.copy())
        self.prev_context_states.append(context_state.copy())

    def get_exploration_stats(self, map_id):
        memory = self.get_current_map_memory(map_id)
        return {
            'visited': len(memory['visited_tiles']),
            'obstructions': len(memory['obstructions']),
        }

In [5]:
# ============================================================================
# CELL 4: Action Selection
# ============================================================================

import random

GBA_ACTIONS = ["UP", "DOWN", "LEFT", "RIGHT", "A", "B", "START", "SELECT"]
ACTION_DELTAS = {"UP": (0, -1), "DOWN": (0, 1), "LEFT": (-1, 0), "RIGHT": (1, 0)}

def anticipatory_action(brain, learning_state, context_state, 
                        exploration_weight=1.3, min_interact_prob=0.15,
                        raw_position=None, forced_explore_prob=0.18):
    """
    Select action based on current state and exploration needs.
    """
    actions_list = brain.actions()
    if not actions_list:
        return Perceptron("action", action="UP", group="move")

    mode = brain.determine_control_mode(context_state, raw_position=raw_position)
    current_map = int(context_state[2])
    
    raw_x = raw_position[0] if raw_position else int(context_state[0] * 255)
    raw_y = raw_position[1] if raw_position else int(context_state[1] * 255)
    
    # Get exploration memory
    memory = brain.get_current_map_memory(current_map)
    visited_tiles = memory['visited_tiles']
    obstructions = memory['obstructions']
    
    # === BUILD ALLOWED ACTIONS LIST ===
    if mode == "battle":
        move_weight = brain.get_group_weight("move")
        interact_weight = brain.get_group_weight("interact")
        total = move_weight + interact_weight + 1e-9
        if random.random() < move_weight / total:
            allowed = [a for a in actions_list if a.group == "move"]
        else:
            allowed = [a for a in actions_list if a.group == "interact"]
    elif mode == "interact":
        allowed = [a for a in actions_list if a.group == "interact"]
    else:  # move
        allowed = [a for a in actions_list if a.group == "move"]

    if not allowed:
        allowed = actions_list

    # === FORCED RANDOM EXPLORATION ===
    if random.random() < forced_explore_prob:
        chosen = random.choice(allowed)
        brain.track_consecutive_action(chosen.action)
        return chosen

    # === SCORE ACTIONS ===
    action_scores = []
    
    for a in allowed:
        # Base score from perceptron prediction
        score = a.utility + random.random() * 0.1
        
        # Movement actions
        if a.group == "move":
            if mode == "move":
                score *= exploration_weight
            
            dx, dy = ACTION_DELTAS.get(a.action, (0, 0))
            target_tile = (raw_x + dx, raw_y + dy)
            
            # Bonus for unvisited tiles
            if target_tile not in visited_tiles:
                score *= brain.UNVISITED_TILE_BONUS
            
            # Penalty for known obstructions
            if target_tile in obstructions:
                score *= brain.OBSTRUCTION_PENALTY
        
        # Interaction actions
        elif a.group == "interact":
            score = max(score, min_interact_prob)
            
            # Reduce Start/Select probability
            if a.action in ['START', 'SELECT']:
                score *= 0.3
        
        action_scores.append((a, score))

    # === SELECT BEST ===
    if action_scores:
        best_action = max(action_scores, key=lambda x: x[1])[0]
        brain.track_consecutive_action(best_action.action)
        return best_action
    
    # Fallback
    fallback = random.choice(actions_list)
    brain.track_consecutive_action(fallback.action)
    return fallback

In [6]:
# ============================================================================
# CELL 5: Main Loop
# ============================================================================

# Initialize brain
brain = Brain()

# Create action perceptrons
for action in ["UP", "DOWN", "LEFT", "RIGHT"]:
    brain.add(Perceptron("action", action=action, group="move"))
for action in ["A", "B", "START", "SELECT"]:
    brain.add(Perceptron("action", action=action, group="interact"))

print("=" * 60)
print("Pokemon FireRed AI - Frame-Based Vision System")
print("=" * 60)
print(f"State file: {STATE_FILE}")
print(f"Action file: {ACTION_FILE}")
print(f"Frame size: {FRAME_DIM} values ({FRAME_SHAPE})")
print("=" * 60)

prev_context_state = None
prev_raw_position = None

# Initial capture config
write_capture_config(1, "startup")

print("\nStarting AI loop... (Ctrl+C to stop)")
print("-" * 60)

try:
    while True:
        loop_start = time.time()
        
        # Read state from bridge
        context_state, frame_state, frame_diff, dead, raw_position = read_game_state()
        
        raw_x, raw_y = raw_position
        in_battle = context_state[3]
        current_map = int(context_state[2])
        current_dir = int(context_state[5])
        
        brain.update_position(raw_x, raw_y)
        
        # Build learning state
        derived = compute_derived_features(context_state, prev_context_state)
        learning_state = build_learning_state(derived, frame_state, in_battle)
        
        brain.log_state(learning_state, context_state)
        
        # Select and execute action
        action = anticipatory_action(
            brain, learning_state, context_state,
            raw_position=raw_position
        )
        
        if action is not None:
            write_action(action.action)
            brain.last_action = action.action
        else:
            write_action("NONE")
        
        # === LOGGING ===
        if brain.timestep % 60 == 0:
            dir_names = {0: "DOWN", 1: "UP", 2: "LEFT", 3: "RIGHT"}
            dir_str = dir_names.get(current_dir, "?")
            mode = brain.control_mode
            stagnation = brain.stagnation_level()
            stats = brain.get_exploration_stats(current_map)
            avg_error = np.mean(list(brain.error_history)[-60:]) if brain.error_history else 0
            avg_frame_diff = np.mean(list(brain.frame_diff_history)[-60:]) if brain.frame_diff_history else 0
            
            print(f"[{brain.timestep:5d}] Pos:({raw_x:3d},{raw_y:3d}) Map:{current_map:3d} "
                  f"Dir:{dir_str:5s} Mode:{mode:8s}")
            print(f"        Visited:{stats['visited']:4d} Obs:{stats['obstructions']:4d} "
                  f"Stag:{stagnation:.2f} CapRate:{brain.capture_rate}")
            print(f"        Error:{avg_error:.4f} FrameDiff:{avg_frame_diff:.4f} "
                  f"Action:{brain.last_action or '-'}")
            print()
        
        # === MILESTONES ===
        if brain.timestep % 500 == 0 and brain.timestep > 0:
            total_visited = sum(len(m['visited_tiles']) for m in brain.exploration_memory.values())
            total_obs = sum(len(m['obstructions']) for m in brain.exploration_memory.values())
            
            print("#" * 60)
            print(f"# MILESTONE {brain.timestep}")
            print(f"# Maps explored: {len(brain.exploration_memory)}")
            print(f"# Total visited: {total_visited} | Obstructions: {total_obs}")
            print(f"# Capture rate: {brain.capture_rate}")
            print("#" * 60 + "\n")
        
        # Sleep to control loop speed
        time.sleep(0.02)
        
        # === LEARNING ===
        next_context, next_frame, next_frame_diff, dead, next_raw_position = read_game_state()
        next_in_battle = next_context[3]
        next_derived = compute_derived_features(next_context, context_state)
        next_learning_state = build_learning_state(next_derived, next_frame, next_in_battle)
        
        brain.learn(
            learning_state, next_learning_state,
            context_state, next_context,
            frame_diff=frame_diff,
            dead=dead,
            raw_position=raw_position
        )
        
        prev_context_state = context_state.copy()
        prev_raw_position = raw_position
        brain.timestep += 1

except KeyboardInterrupt:
    print("\n" + "=" * 60)
    print("Stopping AI...")
    brain.save_exploration_memory()
    print("Exploration memory saved.")
    print("=" * 60)

  Loaded exploration memory: 2 maps
Pokemon FireRed AI - Frame-Based Vision System
State file: /Users/achal/Downloads/PokeTest/game_state.json
Action file: /Users/achal/Downloads/PokeTest/action.json
Frame size: 7200 values ((40, 60, 3))

Starting AI loop... (Ctrl+C to stop)
------------------------------------------------------------
[    0] Pos:( 13, 15) Map: 13 Dir:DOWN  Mode:move    
        Visited:   1 Obs:   0 Stag:0.00 CapRate:1
        Error:0.0000 FrameDiff:0.0000 Action:UP

[   60] Pos:( 13, 15) Map: 13 Dir:DOWN  Mode:move    
        Visited:   1 Obs:   0 Stag:0.98 CapRate:1
        Error:0.3456 FrameDiff:0.0167 Action:DOWN

  🗺️ MAP CHANGE → 0
  🗺️ MAP CHANGE → 13
[  120] Pos:( 13, 15) Map: 13 Dir:DOWN  Mode:interact
        Visited:   1 Obs:   0 Stag:0.98 CapRate:1
        Error:0.6901 FrameDiff:0.0000 Action:B

  🗺️ MAP CHANGE → 0
  🗺️ MAP CHANGE → 13
  🗺️ MAP CHANGE → 0
  🗺️ MAP CHANGE → 13
  🗺️ MAP CHANGE → 0
  🗺️ MAP CHANGE → 13
[  180] Pos:( 13, 16) Map: 13 Dir:RIGHT